### From a velocity set (+labels), build a training set
e.g. from  `velocity_cat=custom_ASlag=45_MS=5.5.npy` build `dataset_cat=custom_ASlag=45_MS=6_c=5_sigIt=8_days=2_minNStat=3.npy`

In [ ]:
import seismicutils
from seismicutils import SeismicUtils
import numpy as np
import pandas as pd
from importlib import reload
import matplotlib.pyplot as plt
if(True):
    reload(seismicutils)
from sklearn.metrics.pairwise import haversine_distances
import os
from mpl_toolkits.axes_grid1 import make_axes_locatable
import networkx as nx
from IPython.display import display, HTML

In [ ]:
pd.set_option('display.max_rows', None)

### Load

In [ ]:
R0 = 6371  ## Earth's Radius
R1 = 200   ## Lateral distance (in km) for which we expect AS to occur.
## we predict aftershocks up to 45 days after the MS:
aftershocks_time_window = np.timedelta64(45,'D') 
starting_date = np.datetime64('1995-01-01')  
min_mainshock_mag = 6
force_filter = True
catalog_type = 'custom'
nDays = 9
name= 'velset/' +  SeismicUtils.format_velset_filename(catalog_type,aftershocks_time_window.astype('int'), min_mainshock_mag, nDays)
fit_dataset = np.load(name, allow_pickle=True).item()


In [ ]:
# the number of days present in the data 
maximum_n_days = fit_dataset[list(fit_dataset.keys())[0]][0].shape[0]
maximum_n_days

### Parameters
crucial parameters are:
- cell_size_km
- sigma_interpolation
- min_aftershock_mag
- n_days_from_end
- min_stations_inside

In [ ]:
# as we looked for stations in a radius of R0=2*150 kms, the maximal size
# of the window should be ~150*sqrt(2) km
# in cell_size_km = 5 means 60 \times 60 image
cell_size_km = 5
cell_size_rads = cell_size_km/R0
cell_size_degs = cell_size_rads*180/np.pi
scale_factor  = 1e3     ## from meters to mm (for the GPS data)
sigma_interpolation = 8 ## expressed in number of cell sizes
min_lateral_size = 25  # box latreal length will be 2 times this
min_aftershock_mag = 4
n_days_from_end = 2     ## you can further restrict the input data number of days
min_stations_inside = 3 # minimal number of stations inside the image

soft_labels = False      ## smoothed labels (from {0,1} to continuous, smoothed over space)
sigma_softlabels = 1 ## expressed in number of cell sizes
regression = False       ## labels are total magnitudes (m corresponding to sum of energies)
use_constrained_fit = True  ## Thin 2D elastic sheet model for interpolating GPS data
# Remark: the alpha_min will be chosen later, at training time.

### Fit

In [ ]:
ids = list(fit_dataset.keys())  ## contains ordered sequences ids
ids.sort()
cartesian_dict = {}
labels_dict = {}
mainshock_info = {}
minimal_distances = {}
for id in ids:
    fd = fit_dataset[id]
    v, o, md,mm, ml, al, am = fd

    

    # locate a square region around the mainshock
    min_lat, max_lat = ml[0]-min_lateral_size*cell_size_degs,ml[0]+min_lateral_size*cell_size_degs
    min_lon, max_lon = ml[1]-min_lateral_size*cell_size_degs,ml[1]+min_lateral_size*cell_size_degs
    n_discr_lat = 2*min_lateral_size
    n_discr_lon = 2*min_lateral_size

    # number of stations inside the region
    n_stations_inside_image = ((o[:, 0] >= min_lat)*(o[:, 0] <= max_lat)\
                              *(o[:, 1] >= min_lon)*(o[:, 1] <= max_lon)).sum()

    # don't proceed if not enough stations
    if(n_stations_inside_image < min_stations_inside):
        print('Skip:', id, 'with # stats:', n_stations_inside_image, 'of total (downloaded)', o.shape[0])
        continue

    od_stats = haversine_distances(o*np.pi/180).flatten()
    od_stats = od_stats[od_stats>0]
    minimal_distances[id] = R0*od_stats.min()

    y_label = np.zeros((n_discr_lat, n_discr_lon))
    # discretize AS locations to rows and columns indices
    aftershocks_row = ((al[:,0]-min_lat)/cell_size_degs).astype('int')
    aftershocks_col = ((al[:,1]-min_lon)/cell_size_degs).astype('int')
    # mask to make sure no event is outside (overkill)
    aftershocks_mask = aftershocks_row < n_discr_lat
    aftershocks_mask *= aftershocks_row >= 0
    aftershocks_mask *= aftershocks_col < n_discr_lon
    aftershocks_mask *= aftershocks_col >= 0 
    aftershocks_mask *= am >= min_aftershock_mag
    
    if(aftershocks_mask.sum() == 0):
        continue
    if(regression):
        continue
    else:
        if(soft_labels):
            SeismicUtils.create_soft_labels(y_label, aftershocks_row[aftershocks_mask], aftershocks_col[aftershocks_mask], cell_size_rads, sigma_softlabels*cell_size_rads)
        else:
            y_label[aftershocks_row[aftershocks_mask], aftershocks_col[aftershocks_mask]] = 1
    
    if (use_constrained_fit):
        v0   = v.mean(axis=1)
        v0_s = v.std (axis=1)
        v_fit = (v - v0[:,None,:])/v0_s[:,None,:]
    try:
        out_arrays = []
        for time_index in range(max(0, v.shape[0] - n_days_from_end),v.shape[0]):
            if(use_constrained_fit):
                automatic_reg_factor = 2 #minimal_distances[id]/cell_size_km

                out_array, body_forces = seismicutils.fit_constrained( 
                    np.pi*o/180,
                    v_fit[time_index,...],
                    n_discr_lat, n_discr_lon,
                    np.pi*min_lat/180,
                    np.pi*min_lon/180,
                    cell_size_rads,
                    sigma_interpolation*cell_size_rads,
                    reg_factor=automatic_reg_factor,
                    index_ratio=0.8)
                print('Success', id, time_index)
                out_array[:,:,:2] *= v0_s[time_index,:2]
                out_array[:,:,:2] += v0[time_index,:2]
            else:
                out_array = np.zeros((n_discr_lat, n_discr_lon, 4))*np.nan
                ## where the real computation happens (using numba, jit)
                seismicutils.fit_gps(out_array, 
                        np.pi*o/180,
                        v[time_index,...],
                        n_discr_lat, n_discr_lon,
                        np.pi*min_lat/180,
                        np.pi*min_lon/180,
                        cell_size_rads,
                        sigma_interpolation*cell_size_rads,
                        min_w=0) # by putting min_w = 0 all the pixels are fit
            out_arrays.append(out_array)
        out_arrays = np.array(out_arrays)
        # out_arrays[:,:,:,:-1] *= scale_factor we rescale later, if needed
        cartesian_dict[id] = out_arrays
        labels_dict[id] = y_label
        mainshock_info[id] = (md, ml)
    except:
        print('Failed', id)
        continue

In [ ]:
if(False):
    direction = 0
    for id in ids:
        if(id in cartesian_dict):
            fd = fit_dataset[id]
            v, o, md,mm, ml, al, am = fd
            iData = cartesian_dict[id]
            print(id)
            fig, ax = plt.subplots(ncols=3, figsize=(16, 4))
            im = ax[0].imshow(scale_factor*iData[-2, :, :, direction], cmap='seismic', interpolation='none')
            ax[0].contour(labels_dict[id])
            divider = make_axes_locatable(ax[0])
            cax = divider.append_axes('right', size='5%', pad=0.05)
            fig.colorbar(im, cax=cax, orientation='vertical')
            ax[0].set_title("%f %f" %(scale_factor*v[-2, :, direction].min(), scale_factor*v[-2, :, direction].max()))
            im = ax[1].imshow(scale_factor*iData[-1, :, :, direction], cmap='seismic', interpolation='none')
            divider = make_axes_locatable(ax[1])
            cax = divider.append_axes('right', size='5%', pad=0.05)
            fig.colorbar(im, cax=cax, orientation='vertical')
            ax[1].set_title("%f %f" %(scale_factor*v[-1, :, direction].min(), scale_factor*v[-1, :, direction].max()))
            ax[2].imshow(iData[0,:,:,-1], vmin=0, vmax=1)
            plt.show()
            #max_i = iData[-1, :, :, :2].max(axis=(0,1))
            #max_v = v[-1, :, :2].max(axis=0)
            #print(id,max_v, max_i,minimal_distances[id]/cell_size_km)

### Data manipulation (basic)

In [ ]:
if(use_constrained_fit):
    n_spatial_components = 2
    n_polar_components = 3 #radial + 2 cos/sin
else:
    n_spatial_components = 3 #include upward direction
    n_polar_components = 5 #radial, cylindrical, 2 cos/sin, 1 sin

#cartesian_data = np.zeros((len(cartesian_dict), n_days_from_end,n_discr_lat, n_discr_lon,  n_spatial_components))
#polar_data = np.zeros((len(cartesian_dict), n_days_from_end,n_discr_lat, n_discr_lon,  n_polar_components))
cartesian_data = []
confidence_data = []
polar_data = []
label_data = []
id_data = []
day_data = []
for id, packed_item in cartesian_dict.items():
    cart_item = packed_item[...,:-1]*scale_factor
    cartesian_data.append(cart_item[None,...])
    
    alpha_item = packed_item[...,-1]
    confidence_data.append(alpha_item[-1])
    
    ## polar coordinates: (2D or 3D)
    north_east_modulus = np.sqrt(cart_item[...,0]**2+cart_item[...,1]**2)
    north_angular = cart_item[...,0]/north_east_modulus ## North/modulus  = opposed/adjacent = cos(angle)
    east_angular  = cart_item[...,1]/north_east_modulus  ## East /modululs = ....   /adjacent = sin(angle)
    if(use_constrained_fit): ## 2D
        temp_polar_concat = np.concatenate([north_east_modulus[...,None]\
                                      ,north_angular[...,None],east_angular[...,None] ], axis=-1)
        ## 0: radial
        ## 1,2: angular
        #polar_feat_names =["north_east_modulus", "north_angular", "east_angular"]
    else: ## 3D
        upward_modulus = np.abs(cart_item[...,2])
        upward_angular = cart_item[...,2]/np.linalg.norm(cart_item, axis=-1)  ## ~sign(up.down)/norm(3 axes)
        temp_polar_concat = np.concatenate([north_east_modulus[...,None], upward_modulus[...,None], north_angular[...,None],\
                                      east_angular[...,None], upward_angular[...,None] ], axis=-1)
        ## 0,1: radial
        ## 2,3,4: angular
        #polar_feat_names =["north_east_modulus", "upward_modulus", "north_angular", "east_angular", "upward_angular"]
    polar_data.append(temp_polar_concat[None,...])
    
    label_data.append(labels_dict[id][None,...])
    id_data.append(id)
    day_data.append(mainshock_info[id][0])

cartesian_data = np.concatenate(cartesian_data)
polar_data = np.concatenate(polar_data)
label_data = np.concatenate(label_data)
confidence_data = np.array(confidence_data)
id_data = np.array(id_data)
day_data = np.array(day_data)
n_days = cartesian_data.shape[1]
# we do this in the same cell to avoid running twice the below lines
# first: we put move the 'time' (index=1) axis close to component axis (index=4)
# second: we flatten the last two axis ('time', 'comp') and we move it in front of (pixel, pixel) axes, adapt to pytorch
cartesian_data  = cartesian_data.transpose((0,2,3,1,4)) 
cartesian_data  = cartesian_data.reshape((cartesian_data.shape[:3] + (-1,))).transpose((0, 3, 1, 2)) 
polar_data = polar_data.transpose((0,2,3,1,4))
polar_data = polar_data.reshape((polar_data.shape[:3] + (-1,))).transpose((0, 3, 1, 2))
# polar_feat_names = still the same

In [ ]:
# feature indices
north_components = np.arange(0, n_days*n_spatial_components, n_spatial_components)
east_components  = np.arange(1, n_days*n_spatial_components, n_spatial_components)
vector_components = np.vstack([north_components,
                               east_components]).T.flatten()

## scalar_components: relate to the cartesian representation (telling us how the object transforms under rotation)
if(use_constrained_fit):  ## n_spatial_components==2
    scalar_components = None
else:
    scalar_components = np.arange(2, n_days*n_spatial_components, n_spatial_components)
    
## compute radial and angular components:
if(use_constrained_fit): ## n_spatial_components==2
    # contains sqrt(n**2+e**2)
    radial_components = np.arange(0, n_days*n_polar_components, n_polar_components)
    # n/sqrt(n**2+e**2) and e/sqrt(n**2+e**2)
    angular_components = np.vstack([np.arange(1, n_days*n_polar_components, n_polar_components), 
                                    np.arange(2, n_days*n_polar_components, n_polar_components)]).T.flatten()
    # no radial/angular 3d, as the system is natively 2D
    radial_3d_components = None
    angular_3d_components = None
#     polar_feat_names =["north_east_modulus", "north_angular", "east_angular"]
    
else:
    # sqrt(n**2+e**2), basically the same as 2D case above
    radial_components = np.arange(0, n_days*n_polar_components, n_polar_components)
    # n/sqrt(n**2+e**2) and e/sqrt(n**2+e**2)
    angular_components = np.vstack([np.arange(2, n_days*n_polar_components, n_polar_components), 
                                    np.arange(3, n_days*n_polar_components, n_polar_components)]).T.flatten()
    radial_3d_components = np.vstack( [np.arange(0, n_days*n_polar_components, n_polar_components),
                                    np.arange(1, n_days*n_polar_components, n_polar_components)]).T.flatten()
    angular_3d_components = np.vstack([np.arange(2, n_days*n_polar_components, n_polar_components), 
                                    np.arange(3, n_days*n_polar_components, n_polar_components),
                                    np.arange(4, n_days*n_polar_components, n_polar_components)]).T.flatten()
#     polar_feat_names =["north_east_modulus", "upward_modulus", "north_angular", "east_angular", "upward_angular"]

day_spatial_components = np.arange(0, n_days*n_spatial_components).reshape(n_days,n_spatial_components)
day_polar_components   = np.arange(0, n_days*n_polar_components).reshape(n_days,n_polar_components)
components_dict = {}
components_dict['north'] = north_components
components_dict['east']  = east_components
components_dict['vector'] = vector_components

## if 3D, otherwise it's None:
components_dict['scalar'] = scalar_components

## good for both 3D and 2D
## in 3D, it gives the 2D subset
## in 2D it is native
components_dict['radial'] = radial_components
components_dict['angular'] = angular_components

## if 3D, otherwise it's None:
components_dict['radial3d'] = radial_3d_components
components_dict['angular3d'] = angular_3d_components

## at the moment not used, however it can be used for visualization
components_dict['day_cart'] = day_spatial_components
components_dict['day_pol'] = day_polar_components

In [ ]:
info = []
for rem_id in id_data:
    fd = fit_dataset[rem_id]
    v, o, md,mm, ml, al, am = fd
    info.append([md, rem_id, mm, ml[0], ml[1], len(am[am>=min_aftershock_mag]) ])
info = pd.DataFrame(info, columns=['day','seq_id', 'mag','lat','lon', 'n_as'])
info

In [ ]:
exclude_distance_km = np.sqrt(2)*cell_size_km*min_lateral_size*2
#drop_ids = set()
#for i in range(0, len(info)):
#    if(info.iloc[i].seq_id in drop_ids):
#        continue
#    for j in range(i+1, len(info)):
#        delta_days = (info.iloc[j].day.to_numpy() - info.iloc[i].day.to_numpy()).astype('timedelta64[D]')
#        if(delta_days > aftershocks_time_window):
#            break
#        arg1 = np.radians(np.array([info.iloc[i].lat, info.iloc[i].lon]))[None,:]
#        arg2 = np.radians(np.array([info.iloc[j].lat, info.iloc[j].lon]))[None,:]
#        d_ij = R0*haversine_distances(arg1, arg2)[0,0]
#        if(d_ij <= exclude_distance_km):
#            print(info.iloc[j].seq_id,delta_days, d_ij, info.iloc[j].day.to_numpy())
#            drop_ids.add(info.iloc[j].seq_id)
#info = info[~info.seq_id.isin(drop_ids)].reset_index(drop=True)

In [ ]:
distance_matrix = R0*haversine_distances(np.radians(info[['lat','lon']].values))
distance_matrix[np.abs(info['day'].values[None,:]-info['day'].values[:,None]) > aftershocks_time_window] =0
distance_matrix[distance_matrix > exclude_distance_km  ] = 0
distance_matrix[np.diag_indices_from(distance_matrix)] = 0

In [ ]:
keep_in_manual_selection = [273, 275, 282, 297, 294, 317, 341, 344,376,378,392,409,422,426, 430,440, 
                            446, 454, 468,472,492,520,521,525,534,537,548]
g = nx.from_numpy_array(distance_matrix)
drop_manual = []
for comp in nx.connected_components(g):
    if(len(comp) > 1):
        comp_df = info[info.index.isin(comp)].reset_index(drop=True)
        comp_df['dt'] = np.insert(np.diff(comp_df.day), 0, 0)
        comp_df['ranking'] = np.argsort(np.argsort(-comp_df['mag'].values))
        comp_df['keep'] = np.zeros(len(comp_df), dtype=int)
        comp_df.loc[comp_df.seq_id.isin(keep_in_manual_selection),'keep']=1
        drop_manual.append(comp_df[~comp_df.seq_id.isin(keep_in_manual_selection)]['seq_id'].values)
        display(comp_df)
        #if(len(comp_df)>20):
        #    plt.scatter(comp_df.lon, comp_df.lat, c=comp_df.ranking, s=10**(comp_df.mag/3))
        #    for _, my_row in comp_df[comp_df.mag>6.5].iterrows():
        #        plt.annotate('%i' % my_row.seq_id, (my_row.lon, my_row.lat), color='red')
        #    plt.show()        
drop_manual = np.concatenate(drop_manual)
print('TO DROP', len(drop_manual), drop_manual)

### Save

In [ ]:
if(not os.path.exists('dataset')):
    os.mkdir('dataset')

In [ ]:
keep_mask = np.ones(len(id_data), dtype=bool)
for to_drop_id in drop_manual:
    keep_mask[id_data==to_drop_id]=False
keep_mask = np.argwhere(keep_mask).flatten()

In [ ]:
scaled_polar = np.copy(polar_data[keep_mask])
for radial_idx in components_dict['radial']:
    sc_min  = np.min(scaled_polar[:, radial_idx,:,:], axis=(1,2))
    sc_max  = np.max(scaled_polar[:, radial_idx,:,:], axis=(1,2))
    scaled_polar[:, radial_idx,:,:] -= sc_min[:,None,None]
    scaled_polar[:, radial_idx,:,:] /= (sc_max-sc_min)[:,None,None]
target_filename = SeismicUtils.format_dataset_filename(catalog_type, aftershocks_time_window.astype('int'), min_mainshock_mag, cell_size_km, sigma_interpolation, n_days_from_end, min_stations_inside, soft_labels, regression, use_constrained_fit)
dict_to_save = {'cartesian' : cartesian_data[keep_mask], \
                'polar' : polar_data[keep_mask], \
                'scpolar' : scaled_polar, \
                'label' : label_data[keep_mask], \
                'alpha' : confidence_data[keep_mask],\
                'id' : id_data[keep_mask], \
                'day' : day_data[keep_mask], \
                'components' : components_dict}
np.save('dataset/' + target_filename, dict_to_save, allow_pickle=True)